In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import DistanceMetric
import math
from math import sin, cos, sqrt, atan2, radians

In [2]:
head, tail = os.path.split(os.getcwd())
data_dir = os.path.join(head, 'data')
data_raw_dir = os.path.join(data_dir, 'raw')
data_raw_dir

'/Users/Gagandeep/Desktop/Concordia/12 Winter 2021/SOEN 471/NoToW/data/raw'

In [3]:
df_spots = pd.read_csv(os.path.join(data_raw_dir, 'Places.csv'), header=0, encoding='cp1252')
df_spots.head()

,sNoPlace,nLongitude,nLatitude,nPositionCentreLongitude,nPositionCentreLatitude,sStatut,sGenre,sType,sAutreTete,sNomRue,nSupVelo,sTypeExploitation,nTarifHoraire,sLocalisation,nTarifMax
0,A007,-73.581856,45.489593,-73.581870,45.489623,1,NORMAL,Double,A008,Sussex,0,Régulier Payez-Partez,350,S,NaN
1,A008,-73.581856,45.489593,-73.581806,45.489592,1,NORMAL,Double,A007,Sussex,0,Régulier Payez-Partez,350,S,NaN
2,A009,-73.581727,45.489532,-73.581740,45.489562,1,NORMAL,Double,A010,Sussex,0,Régulier Payez-Partez,350,S,NaN
3,A010,-73.581727,45.489532,-73.581676,45.489531,1,NORMAL,Double,A009,Sussex,0,Régulier Payez-Partez,350,S,NaN
4,A011,-73.581597,45.489471,-73.581611,45.489501,1,NORMAL,Double,A012,Sussex,0,Régulier Payez-Partez,350,S,NaN


In [4]:
df_spots = df_spots[['nLongitude', 'nLatitude']]
df_spots = df_spots.round(5)
df_spots['nLongitude'] = df_spots['nLongitude'].astype(str)
df_spots['nLatitude'] = df_spots['nLatitude'].astype(str)
df_spots['LOC'] = df_spots['nLatitude']+','+df_spots['nLongitude']
df_spots

,nLongitude,nLatitude,LOC
0,-73.58186,45.48959,"45.48959,-73.58186"
1,-73.58186,45.48959,"45.48959,-73.58186"
2,-73.58173,45.48953,"45.48953,-73.58173"
3,-73.58173,45.48953,"45.48953,-73.58173"
4,-73.5816,45.48947,"45.48947,-73.5816"
...,...,...,...
18647,-73.62571,45.47229,"45.47229,-73.62571"
18648,-73.62562,45.47238,"45.47238,-73.62562"
18649,-73.62522,45.47282,"45.47282,-73.62522"
18650,-73.62518,45.47286,"45.47286,-73.62518"


In [5]:
df_remor = pd.read_csv(os.path.join(data_raw_dir, 'remorquages.csv'), header=0)
df_remor.head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,SECTEUR_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,SECTEUR_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE
0,2015-12-31T00:23:00,-73.587203,45.524981,rue Drolet,NaN,Plateau-Mont-Royal,2015-12-31T00:25:00,-73.586791,45.525131,rue de Grand-Pré,NaN,Plateau-Mont-Royal,Constat d'infraction
1,2015-12-31T00:24:00,-73.622256,45.585753,boulevard des Grandes-Prairies,NaN,Saint-Léonard,2015-12-31T00:30:00,-73.621005,45.586975,boulevard des Grandes-Prairies,NaN,Saint-Léonard,Constat d'infraction
2,2015-12-31T00:25:00,-73.600692,45.535027,rue Saint-Hubert,NaN,Rosemont - La Petite-Patrie,2015-12-31T00:32:00,-73.600881,45.535163,rue Saint-Hubert,NaN,Rosemont - La Petite-Patrie,Constat d'infraction
3,2015-12-31T00:25:00,-73.620824,45.516675,avenue Ducharme,NaN,Outremont,2015-12-31T00:28:00,-73.621995,45.518764,avenue Rockland,NaN,Outremont,Constat d'infraction
4,2015-12-31T00:27:00,-73.580197,45.494309,rue Saint-Mathieu,NaN,Ville-Marie,2015-12-31T00:39:00,-73.579972,45.493133,rue Sainte-Catherine Ouest,NaN,Ville-Marie,Constat d'infraction


In [6]:
df_remor = df_remor[['LONGITUDE_ORIGINE', 'LATITUDE_ORIGINE']]
df_remor = df_remor.round(5)
df_remor['LONGITUDE_ORIGINE'] = df_remor['LONGITUDE_ORIGINE'].astype(str)
df_remor['LATITUDE_ORIGINE'] = df_remor['LATITUDE_ORIGINE'].astype(str)
df_remor['LOC'] = df_remor['LATITUDE_ORIGINE']+','+df_remor['LONGITUDE_ORIGINE']
df_remor

,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,LOC
0,-73.5872,45.52498,"45.52498,-73.5872"
1,-73.62226,45.58575,"45.58575,-73.62226"
2,-73.60069,45.53503,"45.53503,-73.60069"
3,-73.62082,45.51668,"45.51668,-73.62082"
4,-73.5802,45.49431,"45.49431,-73.5802"
...,...,...,...
250072,-73.70518,45.5312,"45.5312,-73.70518"
250073,-73.65453,45.55993,"45.55993,-73.65453"
250074,-73.57954,45.49135,"45.49135,-73.57954"
250075,-73.59578,45.46153,"45.46153,-73.59578"


In [7]:
spot = set(df_spots['LOC'].tolist())
tow = set(df_remor['LOC'].tolist())

In [8]:
len(spot), len(tow)

(11723, 242730)

In [9]:
spot.intersection(tow)

{'45.46228,-73.56698',
 '45.46483,-73.56633',
 '45.48654,-73.56965',
 '45.49749,-73.57553',
 '45.49877,-73.57424',
 '45.4989,-73.55845',
 '45.50007,-73.63133',
 '45.50361,-73.55754',
 '45.50532,-73.57152',
 '45.50943,-73.56178',
 '45.51457,-73.56558',
 '45.52009,-73.55507',
 '45.52121,-73.59457',
 '45.52247,-73.5778',
 '45.52309,-73.61269',
 '45.5265,-73.59806',
 '45.54366,-73.54602'}

In [10]:
tow.intersection(spot)

{'45.46228,-73.56698',
 '45.46483,-73.56633',
 '45.48654,-73.56965',
 '45.49749,-73.57553',
 '45.49877,-73.57424',
 '45.4989,-73.55845',
 '45.50007,-73.63133',
 '45.50361,-73.55754',
 '45.50532,-73.57152',
 '45.50943,-73.56178',
 '45.51457,-73.56558',
 '45.52009,-73.55507',
 '45.52121,-73.59457',
 '45.52247,-73.5778',
 '45.52309,-73.61269',
 '45.5265,-73.59806',
 '45.54366,-73.54602'}

In [12]:
df_spots['nLongitude'] = df_spots['nLongitude'].astype('float64')
df_spots['nLatitude'] = df_spots['nLatitude'].astype('float64')
df_remor['LONGITUDE_ORIGINE'] = df_remor['LONGITUDE_ORIGINE'].astype('float64')
df_remor['LATITUDE_ORIGINE'] = df_remor['LATITUDE_ORIGINE'].astype('float64')

In [18]:
min(df_spots['nLongitude']), min(df_remor['LONGITUDE_ORIGINE'])

(-73.76163, -73.86806)

In [19]:
max(df_spots['nLongitude']), max(df_remor['LONGITUDE_ORIGINE'])

(-73.52986, -73.48005)

In [20]:
min(df_spots['nLatitude']), min(df_remor['LATITUDE_ORIGINE'])

(45.42943, 45.41524)

In [21]:
max(df_spots['nLatitude']), max(df_remor['LATITUDE_ORIGINE'])

(45.56084, 45.70145)

In [23]:
x = np.arange(45.41524, 45.70145, 0.00001, dtype='float64')

In [24]:
y = np.arange(-73.86806, -73.48005, 0.00001, dtype='float64')

In [26]:
x, len(y)

(array([45.41524, 45.41525, 45.41526, ..., 45.70143, 45.70144, 45.70145]),
 38801)

In [28]:
matrix = [x, y]
matrix

[array([45.41524, 45.41525, 45.41526, ..., 45.70143, 45.70144, 45.70145]),
 array([-73.86806, -73.86805, -73.86804, ..., -73.48008, -73.48007,
        -73.48006])]

In [34]:
matrix[0][0]

45.41524

In [36]:
x_indexed = x[np.ix_(x,y)]

IndexError: arrays used as indices must be of integer (or boolean) type